In [ ]:
import boto3
import random

In [ ]:
s3 = boto3.resource('s3')

In [ ]:
contents = """
{"type":"e1", "data":"e1 data"}
{"type":"e2", "data":"e2 data"}
{"type":"e3", "data":"e3 data"}
"""

In [ ]:
contents

In [ ]:
key = 'k-' + str(random.randint(1, 9999999))
print(key)

In [ ]:
s3.Object('be97068west', key).put(Body=contents)

In [ ]:
s3client = boto3.client('s3')

In [ ]:
s3client.put_bucket_versioning(
    Bucket='be97068',
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

### West 2

In [ ]:
import boto3
from botocore.config import Config

In [ ]:
s3west = boto3.client('s3')

In [ ]:
s3west.create_bucket(
    Bucket='be97068west',
    CreateBucketConfiguration={
        'LocationConstraint':'us-west-2'
    }
)

In [ ]:
s3client.put_bucket_versioning(
    Bucket='be97068west',
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

In [ ]:
assume_role_policy_document="""{
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Principal":{
            "Service":"s3.amazonaws.com"
         },
         "Action":"sts:AssumeRole"
      }
   ]
}"""

In [ ]:
iam = boto3.client('iam')
cr = iam.create_role(
    RoleName='repl-sample-role',
    AssumeRolePolicyDocument=assume_role_policy_document
)

print(cr)

In [ ]:
repl_policy_document="""{
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Action":[
            "s3:GetObjectVersionForReplication",
            "s3:GetObjectVersionAcl",
            "s3:GetObjectVersionTagging"
         ],
         "Resource":[
            "arn:aws:s3:::be97068west/*"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ListBucket",
            "s3:GetReplicationConfiguration"
         ],
         "Resource":[
            "arn:aws:s3:::be97068west"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ReplicateObject",
            "s3:ReplicateDelete",
            "s3:ReplicateTags"
         ],
         "Resource":"arn:aws:s3:::be97068/*"
      }
   ]
}"""

In [ ]:
cp = iam.create_policy(
    PolicyName='repl-sample-policy',
    PolicyDocument=repl_policy_document
)

print(cp)

In [ ]:
import os
account_no = os.environ['PRODUCER_ACCOUNT_NO']

In [ ]:
ap = iam.attach_role_policy(
    RoleName='repl-sample-role',
    PolicyArn="arn:aws:iam::{}:policy/repl-sample-policy".format(account_no)
)
print(ap)

In [ ]:
role_arn='arn:aws:iam::{}:role/repl-sample-role'.format(account_no)
print(role_arn)

In [ ]:
# Note to future self: if Prefix is not included you will receive a MalformedXML error with
# no explanation as to the root cause.
s3west.put_bucket_replication(
    Bucket='be97068west',
    ReplicationConfiguration={
        "Role":role_arn,
        "Rules":[
            {
                "ID":"shithead",
                "Status":"Enabled",
                "Destination": {
                    "Bucket" : "arn:aws:s3:::be97068"
                },
                "Prefix":""
            }
        ]
    }
)

In [ ]:
## TODO - ADD CLEAN UP